In [2]:
import fasttext
import fasttext.util
import torch
from torch import nn, tensor
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [4]:
'''fasttext.util.download_model('de', if_exists='ignore')
ft = fasttext.load_model('cc.de.300.bin')'''

In [3]:
train_text_dir = './PHOENIX-2014-T.train.corpus.csv'
test_text_dir  = './PHOENIX-2014-T.test.corpus.csv'
dev_text_dir   = './PHOENIX-2014-T.dev.corpus.csv'
train_text = pd.read_csv(train_text_dir,delimiter='|' )
test_text  = pd.read_csv(test_text_dir,delimiter='|' )
dev_text   = pd.read_csv(dev_text_dir,delimiter='|' )

# print first 5 rows of training dataset text.
print(train_text.shape, test_text.shape, dev_text.shape)
train_text.head(5)['translation']

(7096, 7) (642, 7) (519, 7)


0    und nun die wettervorhersage für morgen donner...
1    mancherorts regnet es auch länger und ergiebig...
2    im nordwesten bleibt es heute nacht meist troc...
3    auch am tag gibt es verbreitet zum teil kräfti...
4    größere wolkenlücken finden sich vor allem im ...
Name: translation, dtype: object

In [4]:
train_list = train_text['translation'].tolist()
test_list  = test_text['translation'].tolist()
dev_list   = dev_text['translation'].tolist()

example_seq = train_list[0]
print("Example string : \n",example_seq)
'''
word_id = [ft.get_word_id(i) for i in example_seq.split(' ')]
print(word_id)

max_length = 56
#from keras.preprocessing import sequence
#padded = sequence.pad_sequences(example_seq, maxlen = max_length)
#padded = [word_id.append(0) for i in range(max_length-len(word_id))]
for i in range(max_length-len(word_id)):
    if len(word_id)<=max_length:
        word_id.append(0)
print("Padded string")
print(word_id)'''

Example string : 
 und nun die wettervorhersage für morgen donnerstag den zwölften august


'\nword_id = [ft.get_word_id(i) for i in example_seq.split(\' \')]\nprint(word_id)\n\nmax_length = 56\n#from keras.preprocessing import sequence\n#padded = sequence.pad_sequences(example_seq, maxlen = max_length)\n#padded = [word_id.append(0) for i in range(max_length-len(word_id))]\nfor i in range(max_length-len(word_id)):\n    if len(word_id)<=max_length:\n        word_id.append(0)\nprint("Padded string")\nprint(word_id)'

#### 0. Use BPEmb instead of fasttext. #### <br>
(since we do not need heavy n-gram models)

In [5]:
from bpemb import BPEmb
bpemb_de = BPEmb(lang="de", vs = 200000, dim = 300)

In [6]:
teststring = train_list[0]
print("given string : ", teststring,'\n')
testBPE = bpemb_de.encode_with_bos_eos(teststring)
print("Byte pair encoded  : ", testBPE,'\n')

ids = bpemb_de.encode_ids_with_bos_eos(teststring)
print("encoded IDs with BOS EOS : ", ids)

vecs = bpemb_de.vectors[ids]
print("Encoded vector with BOS EOS \n",vecs)

decoded = bpemb_de.decode(testBPE)
print("decoded : ", decoded)

print(type(vecs))
testvec = vecs.tolist()[0]

given string :  und nun die wettervorhersage für morgen donnerstag den zwölften august 

Byte pair encoded  :  ['<s>', '▁und', '▁nun', '▁die', '▁wetter', 'vorhersage', '▁für', '▁morgen', '▁donnerstag', '▁den', '▁zwölften', '▁august', '</s>'] 

encoded IDs with BOS EOS :  [1, 43, 1667, 31, 6478, 81915, 137, 5702, 28908, 91, 32966, 897, 2]
Encoded vector with BOS EOS 
 [[ 0.182308 -0.126659 -0.241143 ...  0.318721 -0.168438 -0.062049]
 [-0.120836 -0.031391  0.086993 ...  0.211641 -0.269986  0.115533]
 [-0.205385 -0.032322 -0.212051 ...  0.258928 -0.616859 -0.064941]
 ...
 [ 0.428949 -0.048978  0.039888 ...  0.013891  0.09221  -0.006973]
 [ 0.218135 -0.001797 -0.290565 ...  0.433684 -0.836161 -0.296046]
 [-0.351755  0.015497  0.048089 ... -0.060799 -0.559905 -0.096867]]
decoded :  und nun die wettervorhersage für morgen donnerstag den zwölften august
<class 'numpy.ndarray'>


In [7]:
# we'll pad tokens with '*'
print(bpemb_de.encode_ids('*'))
totalWV    = bpemb_de.vectors
totalWV_VS = np.sqrt(np.sum(totalWV**2, axis = 1))
testvec_VS = np.sqrt(np.sum(vecs**2, axis = 1))
print(totalWV.shape, vecs.shape, totalWV_VS.shape, testvec_VS.shape)
print(totalWV_VS[:10])
norm_mat = np.outer(totalWV_VS, testvec_VS)
print(norm_mat[:10], norm_mat.shape)

[67]
(200000, 300) (13, 300) (200000,) (13,)
[6.7861857 3.9222107 3.6583624 4.9993978 5.829991  5.3909874 4.894021
 5.3435245 6.156014  5.620022 ]
[[26.61685   23.083723  29.130737  26.94188   45.018665  41.945312
  32.13576   39.217175  39.071564  28.290976  38.90293   36.355198
  24.826326 ]
 [15.383737  13.341696  16.836687  15.571593  26.01943   24.243126
  18.573502  22.666344  22.582186  16.35133   22.48472   21.012207
  14.348868 ]
 [14.348868  12.444196  15.704079  14.524088  24.269096  22.612284
  17.324057  21.141573  21.063074  15.251371  20.972166  19.598711
  13.3836155]
 [19.608692  17.005829  21.460678  19.84814   33.165348  30.901203
  23.674486  28.891378  28.784105  20.842022  28.659872  26.782953
  18.289608 ]
 [22.866453  19.831154  25.026125  23.145683  38.675392  36.035088
  27.607733  33.691353  33.566257  24.304688  33.421387  31.232635
  21.328218 ]
 [21.144588  18.33785   23.141636  21.402794  35.763103  33.321613
  25.528845  31.154364  31.038689  22.474524  

**Encoding words are automatic, but decoding must(not a must, but else consumes >15Gb of RAM) be manual.**

In [8]:
def find_similar_vec(vector_arr):
    IDs = []
    score     = totalWV.dot(vector_arr.T)
    vector_VS = np.sqrt(np.sum(vector_arr**2, axis = 1))
    norm_mtx  = np.outer(totalWV_VS, vector_VS)
    score /= norm_mtx
    IDs = np.argmax(score, axis = 0)
    
    IDs[0], IDs[-1] = 1, 2
    return IDs

similar_ids = find_similar_vec(vecs)
print(similar_ids, similar_ids.shape)
print(similar_ids == ids)

[    1    43  1667    31  6478 81915   137  5702 28908    91 32966   897
     2] (13,)
[ True  True  True  True  True  True  True  True  True  True  True  True
  True]


In [9]:
emb_layer = nn.Embedding.from_pretrained(tensor(bpemb_de.vectors))

**5/12** :  We want to make a consistant embedding - that is, equal length emb. through all text datas - for our dataset. Here I represent 4 steps to do preprocessing. <br><br>
#### 1. Extract example string array from textfile, and convert them to ASCII format ####

In [10]:
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io
import time

In [11]:
# train_text.head(5)                                                     # This will show our annota

examplestring = [i for i in test_text.head(10)['translation'].tolist()]
print(examplestring)                                                    # Show strings over row 10. 

['regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar', 'am donnerstag regen in der nordhälfte in der südhälfte mal sonne mal wolken ähnliches wetter dann auch am freitag', 'vom nordmeer zieht ein kräftiges tief heran und bringt uns ab den morgenstunden heftige schneefälle zum teil auch gefrierenden regen', 'sonnig geht es auch ins wochenende samstag ein herrlicher tag mit temperaturen bis siebzehn grad hier im westen', 'deutschland liegt morgen unter hochdruckeinfluss der die wolken weitgehend vertreibt', 'am sonntag im nordwesten eine mischung aus sonne und wolken mit einigen zum teil gewittrigen schauern', 'örtlich schauer oder gewitter die heftig sein können', 'in den nächsten tagen geht es auf jeden fall winterlich weiter immer wieder mal mit etwas schnee', 'und zum wochenende wird es dann sogar wieder ein bisschen kälter', 'auch morgen erwartet uns eine ruhige herbstmischung aus hochnebel wolken und sonne']


In [12]:
def U2A(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')
def preprocess(word):
    #word = U2A(word.lower().strip())   # strip() method erases whitespace in either side of the sentence.
    
    word = word.strip()
    word = '<s> ' + word + ' </s>'
    
    return word

for sentence in examplestring:
    processed_ = preprocess(sentence)
    print(processed_)

<s> regen und schnee lassen an den alpen in der nacht nach im norden und nordosten fallen hier und da schauer sonst ist das klar </s>
<s> am donnerstag regen in der nordhälfte in der südhälfte mal sonne mal wolken ähnliches wetter dann auch am freitag </s>
<s> vom nordmeer zieht ein kräftiges tief heran und bringt uns ab den morgenstunden heftige schneefälle zum teil auch gefrierenden regen </s>
<s> sonnig geht es auch ins wochenende samstag ein herrlicher tag mit temperaturen bis siebzehn grad hier im westen </s>
<s> deutschland liegt morgen unter hochdruckeinfluss der die wolken weitgehend vertreibt </s>
<s> am sonntag im nordwesten eine mischung aus sonne und wolken mit einigen zum teil gewittrigen schauern </s>
<s> örtlich schauer oder gewitter die heftig sein können </s>
<s> in den nächsten tagen geht es auf jeden fall winterlich weiter immer wieder mal mit etwas schnee </s>
<s> und zum wochenende wird es dann sogar wieder ein bisschen kälter </s>
<s> auch morgen erwartet uns eine

#### 2. Now pad the sequences to max_length(maybe 55) ####

In [13]:
# Check the maximum length
max_len = 0
for string in train_list:
    string_ids = bpemb_de.encode_ids_with_bos_eos(string)
    if max_len<=len(string_ids):
        max_len = len(string_ids)
print(max_len)

55


In [14]:
# Check whether encoding-decoding process is valid
idx = 0
for string in test_list:
    sentence_ids = bpemb_de.encode_ids_with_bos_eos(string)
    sentence_vec = bpemb_de.vectors[sentence_ids]
    
    decoded_ids = find_similar_vec(sentence_vec)
    
    if decoded_ids.tolist() != sentence_ids:
        print("Error : ", f'{idx}')
    else:
        print("Done : ", f'{idx}')
    idx+=1

Done :  0
Done :  1
Done :  2
Done :  3
Done :  4
Done :  5
Done :  6
Done :  7
Done :  8
Done :  9
Done :  10
Done :  11
Done :  12
Done :  13
Done :  14
Done :  15
Done :  16
Done :  17
Done :  18
Done :  19
Done :  20
Done :  21
Done :  22
Done :  23
Done :  24
Done :  25
Done :  26
Done :  27
Done :  28
Done :  29
Done :  30
Done :  31
Done :  32
Done :  33
Done :  34
Done :  35
Done :  36
Done :  37
Done :  38
Done :  39
Done :  40
Done :  41
Done :  42
Done :  43
Done :  44
Done :  45
Done :  46
Done :  47
Done :  48
Done :  49
Done :  50
Done :  51
Done :  52
Done :  53
Done :  54
Done :  55
Done :  56
Done :  57
Done :  58
Done :  59
Done :  60
Done :  61
Done :  62
Done :  63
Done :  64
Done :  65
Done :  66
Done :  67
Done :  68
Done :  69
Done :  70
Done :  71
Done :  72
Done :  73
Done :  74
Done :  75
Done :  76
Done :  77
Done :  78
Done :  79
Done :  80
Done :  81
Done :  82
Done :  83
Done :  84
Done :  85
Done :  86
Done :  87
Done :  88
Done :  89
Done :  90
Done :  9